In [ ]:
%pip install -i https://test.pypi.org/simple/ VASA==0.0.4

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from VASA import VASA

In [3]:
# CBG Map of California
counties_map = gpd.GeoDataFrame \
    .from_features(gpd.read_file("../data/ca_cbg")) \
    .set_crs(epsg=4326) \
    .to_crs("+proj=aea +lat_1=20 +lat_2=60 +lat_0=40 +lon_0=-96 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs")

In [4]:
#Lake Fire 2019 Data
fire2019 = pd.read_csv("../data/lakefire2019_data.csv")
fire2019.origin_census_block_group = "0" + fire2019.origin_census_block_group.astype(str)

In [5]:
# Lake Fire Shapefile
lake_fire = gpd.read_file("../data/lake_fire") \
    .set_crs(epsg=4326) \
    .to_crs("+proj=aea +lat_1=20 +lat_2=60 +lat_0=40 +lon_0=-96 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs")

In [6]:
unique_cbgs = np.unique(fire2019.origin_census_block_group)
unique_tracts = np.unique([g[:8] for g in unique_cbgs])
unique_counties = np.unique([g[:5] for g in unique_tracts])

In [7]:
tract_map = counties_map[[(cbg[:8] in unique_tracts) for cbg in counties_map.GEOID]]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

base_map = counties_map[[(cbg[:5] in unique_counties) for cbg in counties_map.GEOID]].copy()

base_map["c"] = "gray"
base_map.loc[[(cbg[:8] in unique_tracts) for cbg in base_map.GEOID], "c"] = "blue"
base_map.loc[[(cbg in unique_cbgs) for cbg in base_map.GEOID], "c"] = "green"

base_map.plot(ax=ax, color=base_map["c"])
lake_fire.plot(ax=ax, color="red")

plt.savefig("fire_weights_filled_in_data.png")
plt.close(fig)

![data levels](fire_weights_filled_in_data.png)

In [8]:
v = VASA(
    df=fire2019[["origin_census_block_group", "date_range_start", "distance_traveled_from_home"]],
    gdf=tract_map,
    df_group_col="origin_census_block_group",
    gdf_group_col="GEOID",
    date_col="date_range_start",
    temp_res="week",
    date_format="%Y-%m-%d %H:%M:%S" # %z
)

In [9]:
del fire2019

In [10]:
v.fill_missing()

In [13]:
fig, ax = plt.subplots(figsize=(10, 10))

v.show_weights_connection(k=15, ax=ax)

plt.savefig("k_weights.png")
plt.close(fig)

/home/alex/Desktop/github/VASA/myenv/lib/python3.8/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
  warnings.warn(message)
/home/alex/Desktop/github/VASA/myenv/lib/python3.8/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 8 disconnected components.
 There are 2 islands with ids: 445, 5105.
  warnings.warn(message)


('WARNING: ', 445, ' is an island (no neighbors)')
('WARNING: ', 5105, ' is an island (no neighbors)')


![](k_weights.png)

In [ ]:
v.show_weights_connection(band=10_000)

In [11]:
fig, ax = plt.subplots(figsize=(10, 10))

v.show_weights_connection(k=10, type="union", ax=ax)

plt.savefig("union_weights.png")
plt.close(fig)

v.W.islands

c:\MOVE\VASA\myenv\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
  warnings.warn(message)
c:\MOVE\VASA\myenv\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 8 disconnected components.
 There are 2 islands with ids: 445, 5105.
  warnings.warn(message)


('WARNING: ', 445, ' is an island (no neighbors)')
('WARNING: ', 5105, ' is an island (no neighbors)')


[]

![](union_weights.png)

In [17]:
fig, ax = plt.subplots(figsize=(10, 10))

v.show_weights_connection(type="queens", ax=ax)

plt.savefig("queens_weights.png")
plt.close(fig)

/home/alex/Desktop/github/VASA/myenv/lib/python3.8/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 8 disconnected components.
 There are 2 islands with ids: 445, 5105.
  warnings.warn(message)


('WARNING: ', 445, ' is an island (no neighbors)')
('WARNING: ', 5105, ' is an island (no neighbors)')


![](queens_weights.png)